In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

#from pylab import rcParams 

from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

import warnings

In [ ]:
def plotar_grafico(var_data, var_x, var_y, var_title, var_media_movel=""):
    plt.figure(figsize=(15,5))

    sns.lineplot(x=var_x, y=var_y, data=var_data[:50], palette="tab10", linewidth=1.5)

    if var_media_movel != "":
        sns.lineplot(x='Data', y= var_media_movel, data= var_data[:50])

    plt.title(var_title)
    plt.xticks(rotation= 30)
    plt.show()

In [ ]:
url_file = 'E:/DADOS/Marcelo/Bootcamp/DS/analise_previsao_series_temporais/dados/A105613189_28_143_208.csv'

In [ ]:
dados = pd.read_csv(url_file, encoding='ISO-8859-1', sep=';', skiprows=3, skipfooter=16, engine='python')
dados.head()

In [ ]:
dados = dados.drop("Total",axis=1)
dados = dados.drop(19,axis=0)

In [ ]:
dados_time_series = dados.melt(id_vars="Ano Diagnóstico", value_name="Casos", var_name="Mes")

dados_time_series['Data'] = dados_time_series["Ano Diagnóstico"] + "-" + dados_time_series["Mes"]


In [ ]:
dados_time_series.head()

In [ ]:
plt.figure(figsize=(15,5))

sns.lineplot(x='Data', y='Casos', data=dados_time_series[:30])

plt.xticks(rotation= 90)
plt.show()

In [ ]:
dados_time_series.info()

In [ ]:
#dados_time_series['data_new'] = dados_time_series['data'].map(str) + '-01'

In [ ]:
meses_dict = {'Jan':'Jan',
                'Fev':'Feb',
                'Mar':'Mar',
                'Abr':'Apr',
                'Mai':'May',
                'Jun':'Jun',
                'Jul':'Jul',
                'Ago':'Aug',
                'Set':'Sep',
                'Out':'Oct',
                'Nov':'Nov',
                'Dez':'Dec'}

In [ ]:
meses_dict_number = {'Jan':1,
                'Feb':2,
                'Mar':3,
                'Apr':4,
                'May':5,
                'Jun':6,
                'Jul':7,
                'Aug':8,
                'Sep':9,
                'Oct':10,
                'Nov':11,
                'Dec':12}

In [ ]:
dados_time_series['Mes'] = dados_time_series['Mes'].map(meses_dict)

In [ ]:
dados_time_series.sample(5)

In [ ]:
dados_time_series['Data'] = dados_time_series["Ano Diagnóstico"] + "-" + dados_time_series["Mes"]

In [ ]:
dados_time_series['Data'] = pd.to_datetime(dados_time_series['Data'])

In [ ]:
#dados_time_series['Mes'] = dados_time_series['Data'].map(dados_time_series['Data'].dt.month)

In [ ]:
dados_time_series['Mes'] = dados_time_series['Mes'].map(meses_dict_number)


In [ ]:
dados_time_series = dados_time_series[['Data','Casos','Mes']]

In [ ]:
plotar_grafico(dados_time_series, 'Data', 'Casos', 'Número de casos de tuberculose - Ano do Diagnóstico')

In [ ]:
#ordenar o dataset pela data
dados_time_series =  dados_time_series.sort_values(by=['Data'])

dados_time_series = dados_time_series.reset_index(drop= True)

In [ ]:
dados_time_series.head()

In [ ]:
dados_time_series['MM_3'] = dados_time_series['Casos'].rolling(3).mean()

In [ ]:
dados_time_series['MM_6'] = dados_time_series['Casos'].rolling(6).mean()

In [ ]:
dados_time_series['MM_3c'] = dados_time_series['Casos'].rolling(3, center= True).mean()

In [ ]:
plotar_grafico(dados_time_series, 'Data', 'Casos', 'Número de casos de tuberculose - Ano do Diagnóstico','MM_3')

In [ ]:
plotar_grafico(dados_time_series, 'Data', 'Casos', 'Número de casos de tuberculose - Ano do Diagnóstico','MM_3c')

In [ ]:
plotar_grafico(dados_time_series, 'Data', 'Casos', 'Número de casos de tuberculose - Ano do Diagnóstico','MM_6')

In [ ]:
sns.lineplot(x='Data', y='Casos', data=dados_time_series[:12])

In [ ]:
sns.lineplot(x='Data', y='Casos', data=dados_time_series[12:24])

In [ ]:
dados_time_series['Ano'] = dados_time_series['Data'].dt.year
dados_time_series['Mes'] = dados_time_series['Data'].dt.month


In [ ]:
sns.lineplot(x='Mes', y='Casos', data=dados_time_series.query("Ano in[2019,2018,2017,2016]"), hue='Ano')

In [ ]:
dados_time_series = dados_time_series.set_index('Data')

In [ ]:
dados_time_series

In [ ]:
decomposicao =  seasonal_decompose(dados_time_series['Casos'])

In [ ]:
decomposicao.plot();


In [ ]:
decomposicao =  seasonal_decompose(dados_time_series['Casos'], model='multiplicative')
decomposicao.plot();

In [ ]:
url_passageiros = 'E:/DADOS/Marcelo/Bootcamp/DS/analise_previsao_series_temporais/dados/AirPassengers.csv'
dados_passengers = pd.read_csv(url_passageiros)
dados_passengers['Month'] =  pd.to_datetime(dados_passengers['Month'])
dados_passengers.info()

In [ ]:
sns.lineplot(x ='Month', y ='#Passengers', data = dados_passengers)

In [ ]:
dados_passengers

In [ ]:
dados_passengers = dados_passengers.set_index('Month')


In [ ]:
decomposicao =  seasonal_decompose(dados_passengers['#Passengers'])
decomposicao.plot();

In [ ]:
decomposicao =  seasonal_decompose(dados_passengers['#Passengers'], model='multiplicative')
decomposicao.plot(2);

In [ ]:
fig, (ax1,ax2,ax3, ax4) = plt.subplots(4,1, figsize=(15,10))
decomposicao.observed.plot(ax=ax1)
ax1.set_ylabel('Observed')
decomposicao.trend.plot(ax=ax2)
ax2.set_ylabel('trend')
decomposicao.seasonal.plot(ax=ax3)
ax3.set_ylabel('seasonal')
decomposicao.resid.plot(ax=ax4)
ax4.set_ylabel('resid')
fig.axes[0].set_title('Decomposição da série de passageiros.')

In [ ]:

rcParams['figure.figsize'] = 11, 9

decomposicao =  seasonal_decompose(dados_passengers['#Passengers'])
fig = decomposicao.plot()
fig.axes[0].set_title('Decomposição da série de passageiros.')

plt.show()

In [ ]:
plot_acf(dados_time_series['Casos']);

In [ ]:
plot_pacf(dados_time_series['Casos']);

In [ ]:
plot_acf(dados_passengers['#Passengers']);


In [ ]:
plot_pacf(dados_passengers['#Passengers']);

## Desafio 01: Transformar os dados em uma série temporal: ANO-MES

## Desafio 02: Converter a coluna data para o tipo datetime

## Desafio 03: Realizar os mesmos processos de transformação dos dados para a base de dados de Hanseníase

## Desafio 04: Deixar o gráfico da série temporal visualmente melhor

## Desafio 05: Analisar outras janelas para média móvel.

## Desafio 06: Realizar sua primeira previsão utilizando médias( toda a série, 3 últimos meses)

In [ ]:
dados_time_series.tail(1)

In [ ]:
media_total =  dados_time_series.tail(1) #dados_time_series.copy()
media_total = media_total.append({'Data': '2020-01-01', 'Casos': dados_time_series['Casos'].mean()}, ignore_index=True)
media_total['Data'] = pd.to_datetime(media_total['Data'])
media_total

In [ ]:
dados_time_series2 = dados_time_series.copy()
dados_time_series2['Casos_Ultimo_Mes'] = dados_time_series2['Casos'].shift()
dados_time_series2['Diff_Casos_Ultimo_Mes'] = dados_time_series2['Casos_Ultimo_Mes'].diff()
dados_time_series2 = dados_time_series2.dropna()
dados_time_series2.head()

In [ ]:
dados_time_series2.tail()

In [ ]:
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [ ]:
#dados_time_series2.query('Data >= "2019-09-01"')['Mes']
dados_time_series2.query('Data >= "2019-08-01"')[dados_time_series2['Mes'] < month]

In [ ]:
mean_error = []
for month in range(9,12):
    train = dados_time_series2.query('Data >= "2019-08-01"')[dados_time_series2['Mes'] < month]
    val = dados_time_series2.query('Data >= "2019-08-01"')[dados_time_series2['Mes'] == month]
    
    p = val['Casos_Ultimo_Mes'].values
    
    error = rmsle(val['Casos'].values, p)
    print('Mes %d - Erro %.5f' % (month, error))
    mean_error.append(error)
print('Média Erro = %.5f' % np.mean(mean_error))

In [ ]:
train

In [ ]:
val

In [ ]:
plt.figure(figsize=(15,5))

sns.lineplot(x='Data', y='Casos', data=dados_time_series, palette="tab10", linewidth=1.5)
plt.plot(media_total.Data, media_total.Casos, 'r--')

plt.title('Previsão do próximo mês com uso de médias móveis - Pela média total')
plt.xticks(rotation= 30)
plt.show()

In [ ]:
media = dados_time_series['Casos'].tail(3).mean()

media_ultimos_meses = dados_time_series.tail(1)
media_ultimos_meses = media_ultimos_meses.append({'Data': '2020-01-01', 'Casos': media}, ignore_index=True)
media_ultimos_meses['Data'] = pd.to_datetime(media_ultimos_meses['Data'])
media_ultimos_meses

In [ ]:
plt.figure(figsize=(15,5))

sns.lineplot(x='Data', y='Casos', data=dados_time_series, palette="tab10", linewidth=1.5)
plt.plot(media_ultimos_meses.Data, media_ultimos_meses.Casos, 'r--')

plt.title('Previsão do próximo mês com uso de médias móveis - Últimos 3 mêses')
plt.xticks(rotation= 30)
plt.show()

## Desafio 07: Transformar as datas em index e realizar os slices pelas datas

## Desafio 08: Formatar os gráficos de decomposição, colocar título e modificar os tamanhos. Realizar a decomposição de outras séries temporais.

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 11, 9

decomposicao =  seasonal_decompose(dados_passengers['#Passengers'])
fig = decomposicao.plot()
fig.axes[0].set_title('Decomposição da série de passageiros.')

plt.show()

## Desafio 09: Realizar uma análise de autocorrelação e autocorrelação parcial dos dados de COVID-19